In [4]:
from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import random as rand
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import torch.optim as optim
import torch.nn.functional as F

# Global Parameters:
BATCH_SIZE = 10
IS_CUDA    = False
LR         = 0.03
MOMENTUM   = 1e-6
EPOCHS      = 10
LOG_IN     = 50

# Load Dataset
def load_dataset(txt_filepath):
    
    # PLace to load the informaton we have
    #features = np.loadtxt(txt_filepath).T
    #data = []
    #target = []
    #for i in range(1,7):
    #    for _ in range(200):
    #        data.append(np.random.randint(5*i, size=20))
    #         target.append(i)
    data = np.random.randint(5, size=(1200, 20))
    target = np.random.randint(6, size=1200)
    return np.sort(data[:]), np.sort(target)

# Global objects:
class PokerDataset(Dataset):

    def __init__(self, txt_filepath):
        self.data, self.target = load_dataset(txt_filepath)
    def __getitem__(self, index):
        return self.data[index,:], self.target[index] 
    def __len__(self):
        return self.data.shape[0]

# Network Architecture:
class BlueNet_all(nn.Module):

    def __init__(self):
        super(BlueNet_all, self).__init__()
        self.layer1 = nn.Linear(20, 40)
        self.layer2 = nn.Linear(40, 120)
        self.layer3 = nn.Linear(120, 60)
        self.layer4 = nn.Linear(60, 30)
        self.layer5 = nn.Linear(30, 6)

        self.relu = nn.ReLU()
        self.softplus = nn.Softplus()

    def forward(self, x):
        fc1 = F.relu(self.layer2(self.relu((self.layer1(x)))))
        out_x = self.layer5(self.relu(self.layer4(self.relu((self.layer3(fc1))))))
        return out_x

train_set = PokerDataset(0)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

model = BlueNet_all()
if IS_CUDA:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=LR)

# Use MSE as Reconstrcution Loss
criterion = nn.CrossEntropyLoss()

def train(epoch):

    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        # reshape to vector
        #data = data.view(data.size(0), -1)
        # move to cuda if available
        if IS_CUDA:
            data = data.cuda()
        # convert to Variable
        data = Variable(data.type(torch.FloatTensor))
        target = Variable(target.type(torch.LongTensor))
        # forward: evaluate with model
        output = model(data)
        loss = criterion(output, target)
        # backward: compute gradient and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_idx % LOG_IN == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

best_acc = 0
# run training
for epoch in range(EPOCHS):
    train(epoch)
    #test_loss = test(show_plot=epoch%10 == 1)
    #if test_loss < best_loss:
    #    best_loss = test_loss
    #    if args.is_noisy:
    #        torch.save(model.state_dict(), 'best_fc_denoising_autoencoder.pth')
    #    else:
    #        torch.save(model.state_dict(), 'best_fc_autoencoder.pth')

Train Epoch: 0 [0/1200 (0%)]	Loss: 1.793135
Train Epoch: 0 [500/1200 (42%)]	Loss: 1.786929
Train Epoch: 0 [1000/1200 (83%)]	Loss: 1.796950
Train Epoch: 1 [0/1200 (0%)]	Loss: 1.790215
Train Epoch: 1 [500/1200 (42%)]	Loss: 1.778614
Train Epoch: 1 [1000/1200 (83%)]	Loss: 1.805952
Train Epoch: 2 [0/1200 (0%)]	Loss: 1.779331
Train Epoch: 2 [500/1200 (42%)]	Loss: 1.772827
Train Epoch: 2 [1000/1200 (83%)]	Loss: 1.804612
Train Epoch: 3 [0/1200 (0%)]	Loss: 1.799015
Train Epoch: 3 [500/1200 (42%)]	Loss: 1.786619
Train Epoch: 3 [1000/1200 (83%)]	Loss: 1.789544
Train Epoch: 4 [0/1200 (0%)]	Loss: 1.776237
Train Epoch: 4 [500/1200 (42%)]	Loss: 1.798314
Train Epoch: 4 [1000/1200 (83%)]	Loss: 1.784218
Train Epoch: 5 [0/1200 (0%)]	Loss: 1.783657
Train Epoch: 5 [500/1200 (42%)]	Loss: 1.787669
Train Epoch: 5 [1000/1200 (83%)]	Loss: 1.792049
Train Epoch: 6 [0/1200 (0%)]	Loss: 1.800703
Train Epoch: 6 [500/1200 (42%)]	Loss: 1.800264
Train Epoch: 6 [1000/1200 (83%)]	Loss: 1.778535
Train Epoch: 7 [0/1200 (0%)

In [2]:
import random
random.shuffle(random.choice(range(100000)))

TypeError: object of type 'int' has no len()